# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

Data

In [3]:
df = pd.read_csv("Dataframes/analysts.csv")
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,3.625620,7.0,3.00000,0.00000,3.000,36.00000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.482,2.046169,7.0,2.87500,-0.12500,3.000,42.57143,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.551601,8.0,2.87500,0.00000,2.875,45.00000,4.30946
3,AVY.N,2013-10-01,Materials,8.095,8.095,2.775994,7.0,2.62500,-0.25000,2.875,47.71429,3.45230
4,AVY.N,2014-01-01,Materials,1.471,1.471,3.204412,8.0,2.33333,-0.29167,2.625,54.62500,3.42555
...,...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,2.836295,8.0,2.30000,0.00000,2.300,545.28571,50.48823
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,3.269867,8.0,2.30000,0.00000,2.300,571.00000,41.33833
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,6.630736,9.0,2.00000,-0.30000,2.300,519.88889,37.49206
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,3.625249,9.0,2.20000,0.20000,2.000,435.37500,57.57373


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [5]:
# Calculate the 0.15th and 99.85th percentile
pct_0_15 = np.percentile(df['Earnings Per Share - Actual Surprise'], 0.5)
pct_99_85 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99.5)

# Remove the data points outside the 0.15th and 99.85th percentile
df_clean = df[(df['Earnings Per Share - Actual Surprise'] >= pct_0_15) & (df['Earnings Per Share - Actual Surprise'] <= pct_99_85)].copy()

Standardising

In [6]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Earnings Per Share – Coefficient of Variation"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share – Coefficient of Variation"]).reshape(-1,1))
rescale["Number of Analysts"] = MinMaxScaler().fit_transform(np.array(rescale["Number of Analysts"]).reshape(-1,1))
rescale["Recommendation change"] = MinMaxScaler().fit_transform(np.array(rescale["Recommendation change"]).reshape(-1,1))
rescale["Price Target - Mean"] = MinMaxScaler().fit_transform(np.array(rescale["Price Target - Mean"]).reshape(-1,1))
rescale["Price Target - Standard Deviation"] = MinMaxScaler().fit_transform(np.array(rescale["Price Target - Standard Deviation"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.534474,0.125000,3.00000,0.538462,3.000,0.005786,0.002079
1,AVY.N,2013-04-01,Materials,2.482,2.482,0.533129,0.125000,2.87500,0.480770,3.000,0.006891,0.003330
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.532708,0.145833,2.87500,0.538462,2.875,0.007299,0.003658
3,AVY.N,2013-10-01,Materials,8.095,8.095,0.533751,0.125000,2.62500,0.423078,2.875,0.007756,0.002930
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.534116,0.145833,2.33333,0.403846,2.625,0.008917,0.002908
...,...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.533802,0.145833,2.30000,0.538462,2.300,0.091403,0.042855
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.534171,0.145833,2.30000,0.538462,2.300,0.095726,0.035088
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.537034,0.166667,2.00000,0.400001,2.300,0.087134,0.031824
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.534474,0.166667,2.20000,0.630770,2.000,0.072926,0.048869


In [7]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [8]:
#reformatting indices of dataframe for panel regression
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share - Actual Surprise  \
Instrument Date                                               
AVY.N      2013-01-01                                11.178   
           2013-04-01                                 2.482   
           2013-07-01                                 1.068   
           2013-10-01                                 8.095   
           2014-01-01                                 1.471   
...                                                     ...   
POOL.OQ    2021-10-01                                17.194   
           2022-01-01                                40.267   
           2022-04-01                                34.342   
           2022-07-01                                 1.503   
           2022-10-01                                 4.196   

                       Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
AVY.N      2013-01-01                                        11.178   
           2013-04-01                                         2.482   
           2013-07-01                                         1.068   
           2013-10-01                                         8.095   
           2014-01-01                                         1.471   
...                                                             ...   
POOL.OQ    2021-10-01                                        17.194   
           2022-01-01                                        40.267   
           2022-04-01                                        34.342   
           2022-07-01                                         1.503   
           2022-10-01                                         4.196   

                       Earnings Per Share – Coefficient of Variation  \
Instrument Date                                                        
AVY.N      2013-01-01                                       0.534474   
           2013-04-01                                       0.533129   
           2013-07-01                                       0.532708   
           2013-10-01                                       0.533751   
           2014-01-01                                       0.534116   
...                                                              ...   
POOL.OQ    2021-10-01                                       0.533802   
           2022-01-01                                       0.534171   
           2022-04-01                                       0.537034   
           2022-07-01                                       0.534474   
           2022-10-01                                       0.534660   

                       Number of Analysts  Recommendation - Mean (1-5)  \
Instrument Date                                                          
AVY.N      2013-01-01            0.125000                      3.00000   
           2013-04-01            0.125000                      2.87500   
           2013-07-01            0.145833                      2.87500   
           2013-10-01            0.125000                      2.62500   
           2014-01-01            0.145833                      2.33333   
...                                   ...                          ...   
POOL.OQ    2021-10-01            0.145833                      2.30000   
           2022-01-01   

FE Panel Regression with **Absolute** Surprise Values

In [9]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Recommendation change":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0041
Estimator:,PanelOLS,R-squared (Between):,0.3429
No. Observations:,18037,R-squared (Within):,0.0015
Date:,"Sat, Mar 04 2023",R-squared (Overall):,0.1593
Time:,01:07:37,Log-likelihood,-8.046e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,17.930
Entities:,502,P-value,0.0000
Avg Obs:,35.930,Distribution:,"F(4,17494)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,17.930


By Industry: example tech industry

In [11]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Recommendation change":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_56074/1826274747.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_56074/1826274747.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0319
Estimator:,PanelOLS,R-squared (Between):,-0.3153
No. Observations:,603,R-squared (Within):,0.0133
Date:,"Sat, Mar 04 2023",R-squared (Overall):,-0.1263
Time:,01:08:21,Log-likelihood,-2187.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.4529
Entities:,19,P-value,0.0015
Avg Obs:,31.737,Distribution:,"F(4,541)"
Min Obs:,1.0000,,
Max Obs:,40.000,F-statistic (robust):,4.4529
